<a href="https://colab.research.google.com/github/YoheiFukuhara/nlp100-2020/blob/main/08.%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88/71_%E5%8D%98%E5%B1%A4%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E3%81%AB%E3%82%88%E3%82%8B%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

問題70で保存した行列を読み込み，学習データについて以下の計算を実行せよ．

```math
\hat{\boldsymbol y_1} = {\rm softmax}(\boldsymbol x_1 W), \\
\hat{Y} = {\rm softmax}(X_{[1:4]} W)
```

ただし，${\rm softmax}$はソフトマックス関数，$X_{[1:4]} \in \mathbb{R}^{4 \times d}$は特徴ベクトル$\boldsymbol x_1, \boldsymbol x_2, \boldsymbol x_3, \boldsymbol x_4$を縦に並べた行列である．

```math
X_{[1:4]} = \begin{pmatrix} 
  \boldsymbol x_1 \\ 
  \boldsymbol x_2 \\ 
  \boldsymbol x_3 \\ 
  \boldsymbol x_4 \\ 
\end{pmatrix}
```

行列$W \in \mathbb{R}^{d \times L}$は単層ニューラルネットワークの重み行列で，ここではランダムな値で初期化すればよい（問題73以降で学習して求める）．なお，$\hat{\boldsymbol y_1} \in \mathbb{N}^L$は未学習の行列$W$で事例$x_1$を分類したときに，各カテゴリに属する確率を表すベクトルである．
同様に，$\hat{Y} \in \mathbb{N}^{n \times L}$は，学習データの事例$x_1, x_2, x_3, x_4$について，各カテゴリに属する確率を行列として表現している．

In [2]:
import tensorflow as tf
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!python --version
!pip show google tensorflow

Python 3.7.12
Name: google
Version: 2.0.3
Summary: Python bindings to the Google search engine.
Home-page: http://breakingcode.wordpress.com/
Author: Mario Vilas
Author-email: mvilas@gmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: beautifulsoup4
Required-by: 
---
Name: tensorflow
Version: 2.6.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: tensorboard, grpcio, wheel, absl-py, wrapt, termcolor, typing-extensions, protobuf, six, google-pasta, clang, tensorflow-estimator, keras-preprocessing, opt-einsum, numpy, gast, flatbuffers, h5py, keras, astunparse
Required-by: kapre


In [4]:
BASE_PATH = '/content/drive/MyDrive/ColabNotebooks/ML/NLP100_2020/08.NeuralNetworks/'
train_raw = tf.data.TFRecordDataset(BASE_PATH+'train.tfrecord')
print(train_raw)

<TFRecordDatasetV2 shapes: (), types: tf.string>


In [5]:
def _parse_function(example_proto):
    # 特徴の記述
    feature_description = {
        'title': tf.io.FixedLenFeature([], tf.string),
        'category': tf.io.FixedLenFeature([], tf.string)}
  
  # 上記の記述を使って入力の tf.Example を処理
    features = tf.io.parse_single_example(example_proto, feature_description)
    X = tf.io.decode_raw(features['title'], tf.float32)
    y = tf.io.decode_raw(features['category'], tf.int32)
    return X, y

In [6]:
dense_layer = tf.keras.layers.Dense(4, activation='softmax', use_bias=False, input_dim=300, kernel_initializer='random_uniform')

for X, y in train_raw.map(_parse_function).batch(4).take(1):

    # 元データ
    print(X.numpy())
    print(y.numpy())  # 使わないけど確認だけする

    print('---出力データ---')
    print(dense_layer(X))

[[ 0.08366042  0.00563401  0.00681481 ...  0.07512489  0.04330679
  -0.08679669]
 [ 0.02719116  0.02655029 -0.0947113  ... -0.10462952 -0.04891968
  -0.00921631]
 [ 0.05766805 -0.01586914 -0.07795715 ... -0.0420634   0.12289429
   0.08764648]
 [-0.05554669  0.04959811  0.06200233 ... -0.01362493  0.03897799
  -0.02062988]]
[[1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]]
---出力データ---
tf.Tensor(
[[0.2518806  0.24666102 0.24710031 0.25435802]
 [0.248636   0.25726244 0.24938346 0.2447181 ]
 [0.25629058 0.24833342 0.24286988 0.25250617]
 [0.25795886 0.23971929 0.23602764 0.26629415]], shape=(4, 4), dtype=float32)
